In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(RCurl)
options(repr.matrix.max.rows = 6)
set.seed(1)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 1.0.0
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.3     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.1.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────────────────────────────────

In [3]:
x <- getURL("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-06/forest.csv")
forest_data <- read.csv(text = x)
forest_data

entity,code,year,net_forest_conversion
<chr>,<chr>,<int>,<dbl>
Algeria,DZA,1990,-8800
Algeria,DZA,2000,33900
Algeria,DZA,2010,7600
⋮,⋮,⋮,⋮
Zimbabwe,ZWE,2000,-46070
Zimbabwe,ZWE,2010,-46070
Zimbabwe,ZWE,2015,-46070


In [4]:
x <- getURL("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-06/forest_area.csv")
forest_area_data <- read.csv(text = x)
forest_area_data

entity,code,year,forest_area
<chr>,<chr>,<int>,<dbl>
Afghanistan,AFG,1990,0.02852494
Afghanistan,AFG,1991,0.02857782
Afghanistan,AFG,1992,0.02863089
⋮,⋮,⋮,⋮
Zimbabwe,ZWE,2018,0.4309932
Zimbabwe,ZWE,2019,0.4303885
Zimbabwe,ZWE,2020,0.4297823


In [5]:
x <- getURL("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-06/brazil_loss.csv")
brazil_loss_data <- read.csv(text = x)
brazil_loss_data

entity,code,year,commercial_crops,flooding_due_to_dams,natural_disturbances,pasture,selective_logging,fire,mining,other_infrastructure,roads,tree_plantations_including_palm,small_scale_clearing
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Brazil,BRA,2001,280000,0,0,1520000,96000,26000,9000,9000,13000,44000,249000
Brazil,BRA,2002,415000,79000,35000,2568000,96000,114000,9000,13000,31000,44000,293000
Brazil,BRA,2003,550000,0,35000,2761000,149000,44000,0,9000,35000,26000,358000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Brazil,BRA,2011,52000,17000,87000,738000,44000,122000,0,0,9000,26000,271000
Brazil,BRA,2012,118000,17000,52000,546000,166000,70000,0,13000,31000,26000,232000
Brazil,BRA,2013,87000,0,13000,695000,131000,26000,35000,4000,17000,35000,284000


In [6]:
x <- getURL("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-06/soybean_use.csv")
soybean_use_data <- read.csv(text = x)
soybean_use_data

entity,code,year,human_food,animal_feed,processed
<chr>,<chr>,<int>,<int>,<int>,<int>
Africa,NA,1961,33000,6000,14000
Africa,NA,1962,43000,7000,17000
Africa,NA,1963,31000,7000,5000
⋮,⋮,⋮,⋮,⋮,⋮
Zimbabwe,ZWE,2011,8000,NA,75000
Zimbabwe,ZWE,2012,8000,NA,87000
Zimbabwe,ZWE,2013,8000,NA,91000


In [7]:
x <- getURL("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-06/vegetable_oil.csv")
vegetable_oil_data <- read.csv(text = x)
vegetable_oil_data

entity,code,year,crop_oil,production
<chr>,<chr>,<int>,<chr>,<int>
Afghanistan,AFG,1961,Soybean,NA
Afghanistan,AFG,1961,Sesame,2253
Afghanistan,AFG,1961,Linseed,3531
⋮,⋮,⋮,⋮,⋮
Zimbabwe,ZWE,1973,Sunflower,666
Zimbabwe,ZWE,1973,Maize,5400
Zimbabwe,ZWE,1973,Palm kernel,NA
